In [ ]:
import os
os.environ["KAGGLE_CONFIG_DIR"]= "/content"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection

In [ ]:
!unzip \*.zip && rm *.zip

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob
import cv2
import imutils
import seaborn as sns
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from keras.models import Model, Sequential
import keras
from PIL import Image

In [ ]:
#Count images in the two folders: yes (Tumor) and no (Healthey)
root =  '/content/brain_tumor_dataset'
dict_img = {}
for dir in os.listdir(root):
  dict_img[dir] = len(os.listdir(os.path.join(root, dir)))
dict_img


In [ ]:
# Define paths to the 'yes' and 'no' folders containing MRI images
yes_path = 'yes'
no_path = 'no'

# Plot some MRI images
plt.figure(figsize=(10, 10))

# Plot the first 'yes' image
img_path = os.path.join(yes_path, os.listdir(yes_path)[0])
img = Image.open(img_path)
plt.subplot(2, 2, 1)
plt.imshow(img)
plt.title('yes')

# Plot the second 'yes' image
img_path = os.path.join(yes_path, os.listdir(yes_path)[1])
img = Image.open(img_path)
plt.subplot(2, 2, 2)
plt.imshow(img)
plt.title('yes')

# Plot the first 'no' image
img_path = os.path.join(no_path, os.listdir(no_path)[0])
img = Image.open(img_path)
plt.subplot(2, 2, 3)
plt.imshow(img)
plt.title('no')

# Plot the second 'no' image
img_path = os.path.join(no_path, os.listdir(no_path)[1])
img = Image.open(img_path)
plt.subplot(2, 2, 4)
plt.imshow(img)
plt.title('no')

plt.show()


In [ ]:
#Define a function for image augmentation
def augment_data(file_dir, n_generated_samples, save_to_dir):
    data_gen = ImageDataGenerator(rotation_range=10,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.1,
                                  brightness_range=(0.3, 1.0),
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='nearest',
                                  rescale= 1/255
                                 )

    for filename in os.listdir(file_dir):
        image = cv2.imread(file_dir + '/' + filename)
        # reshape the image
        image = image.reshape((1,)+image.shape)
        save_prefix = 'aug_' + filename[:-4]
        i=0
        for batch in data_gen.flow(x=image, batch_size=1, save_to_dir=save_to_dir,save_prefix=save_prefix, save_format='jpg'):
                i += 1
                if i > n_generated_samples:
                    break

In [ ]:
#Create augmented images folders
#L = ['aug_train', 'aug_test', 'aug_val']
name = 'aug_data'
if not os.path.exists("./"+name):
  os.mkdir("./"+name)
  for dir in os.listdir(root):
    os.makedirs("./"+name+"/"+dir)
else:
  print(f"{name} Already exists")

In [ ]:
#Augment data for the examples with the label 'yes' in the training set
augment_data(file_dir='./brain_tumor_dataset/yes',n_generated_samples=8, save_to_dir='./aug_data/yes')
#Augment data for the examples with the label 'no' in the training set
augment_data(file_dir='./brain_tumor_dataset/no', n_generated_samples=12, save_to_dir='./aug_data/no')

In [ ]:
#Count images in the two folders: yes (Tumor) and no (Healthy) in the folder of augmented images
root =  '/content/aug_data'
dict_img = {}
for dir in os.listdir(root):
  dict_img[dir] = len(os.listdir(os.path.join(root, dir)))
dict_img

In [ ]:
#Define a function that creates new folders for the train, test and val sets and append random pictures to them based on the split percentage
def create_folders(name, perc):
  if not os.path.exists("./"+name):
    os.mkdir("./"+name)
    for dir in os.listdir(root):
      os.makedirs("./"+name+"/"+dir)
      for img in np.random.choice(a=os.listdir(os.path.join(root, dir)), size=(math.floor(perc*dict_img[dir])), replace=False):
        Src = os.path.join(root, dir, img)
        Dest = os.path.join("./"+name, dir)
        shutil.copy(Src, Dest)
        os.remove(Src)
  else:
    print(f"{name} Already exists")

In [ ]:
#Create the training set
create_folders('train', 0.7)

In [ ]:
#Create the test set
create_folders('test', 0.15)

In [ ]:
#Create the validation set
create_folders('val', 0.15)

In [ ]:
#Define a function that counts images in the folders: yes (Tumor) and no (Healthey)
def count_img(folder):
  dict_img = {}
  for dir in os.listdir(folder):
    dict_img[dir] = len(os.listdir(os.path.join(folder, dir)))
  return dict_img

In [ ]:
#Count images in the training set
count_img('train')

In [ ]:
#Count images in the test set
count_img('test')


In [ ]:
#Count images in the validation set
count_img('val')

In [ ]:
#Define a function that crop tha brain contour
def crop_brain_contour(image, plot=False):

    #Convert the image to grayscale, and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    #Find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)

    #Extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])

    #Crop new image out of the original image using the four extreme points (left, right, top, bottom)
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]

    if plot:
        plt.figure()
        plt.subplot(1, 2, 1)
        plt.imshow(image)
        plt.tick_params(axis='both', which='both', top=False, bottom=False, left=False, right=False,labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        plt.title('Original Image')
        plt.subplot(1, 2, 2)
        plt.imshow(new_image)
        plt.tick_params(axis='both', which='both',top=False, bottom=False, left=False, right=False,labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        plt.title('Cropped Image')
        plt.show()

    return new_image

In [ ]:
#Example
ex_img = cv2.imread('no/1 no.jpeg')
img = crop_brain_contour(ex_img, plot=True)

In [ ]:
#Define a function that load data
def load_data(dir_list, image_size):

    # load all images in a directory
    X = []
    y = []
    image_width, image_height = image_size

    for directory in dir_list:
        for filename in os.listdir(directory):
            image = cv2.imread(directory+'/'+filename)
            image = crop_brain_contour(image, plot=False)
            image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
            # normalize values
            image = image / 255.
            # convert image to numpy array and append it to X
            X.append(image)
            # append a value of 1 to the target array if the image
            # is in the folder named 'yes', otherwise append 0.
            if directory[-3:] == 'yes':
                y.append([1])
            else:
                y.append([0])

    X = np.array(X)
    y = np.array(y)

    # Shuffle the data
    X, y = shuffle(X, y)

    print(f'Number of examples is: {len(X)}')
    print(f'X shape is: {X.shape}')
    print(f'y shape is: {y.shape}')

    return X, y

In [ ]:
#Load trainig data
IMG_WIDTH, IMG_HEIGHT = (240, 240)
X_train, y_train = load_data(['/content/train/yes', '/content/train/no'], (IMG_WIDTH, IMG_HEIGHT))

In [ ]:
#Define a function that plot images
def plot_sample_images(X, y, n=40):
    for label in [0,1]:
        # grab the first n images with the corresponding y values equal to label
        images = X[np.argwhere(y == label)]
        n_images = images[:n]

        columns_n = 10
        rows_n = int(n/ columns_n)

        plt.figure(figsize=(10,4))

        i = 1 # current plot
        for image in n_images:
            plt.subplot(rows_n, columns_n, i)
            plt.imshow(image[0])

            # remove ticks
            plt.tick_params(axis='both', which='both',
                            top=False, bottom=False, left=False, right=False,
                           labelbottom=False, labeltop=False, labelleft=False, labelright=False)

            i += 1

        label_to_str = lambda label: "Yes" if label == 1 else "No"
        plt.suptitle(f"Brain Tumor: {label_to_str(label)}")
        plt.show()


In [ ]:
#Plot samples from the training set
plot_sample_images(X_train, y_train)

In [ ]:
#Load test data
IMG_WIDTH, IMG_HEIGHT = (240, 240)
X_test, y_test = load_data(['/content/test/yes', '/content/test/no'], (IMG_WIDTH, IMG_HEIGHT))

In [ ]:
#Plot samples from the testing set
plot_sample_images(X_test, y_test)

In [ ]:
#Load validation data
IMG_WIDTH, IMG_HEIGHT = (240, 240)
X_val, y_val = load_data(['/content/val/yes', '/content/val/no'], (IMG_WIDTH, IMG_HEIGHT))

In [ ]:
#Plot samples from the validation set
plot_sample_images(X_val, y_val)

In [ ]:
#Build our model
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', input_shape = (240, 240, 3)))

model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Dropout(rate = 0.25))

model.add(Flatten())
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0.25))
model.add(Dense(units = 1, activation = 'sigmoid'))
#Compile our model
model.compile(optimizer = 'adam', loss = keras.losses.binary_crossentropy, metrics = ['accuracy'])
model.summary()

In [ ]:
#Early stopping and model checkpoint
from keras.callbacks import ModelCheckpoint, EarlyStopping

es = EarlyStopping(monitor = 'val_accuracy', min_delta = 0.01, patience = 5, verbose = 1, mode = 'auto')
mc = ModelCheckpoint(monitor ='val_accuracy', filepath = './bestmodel.h5', verbose = 1, save_best_only = True, mode = 'auto')

cd = [es, mc]

In [ ]:
#Train our model
hist = model.fit(x = X_train, y = y_train, batch_size = 32, epochs = 30, validation_data = (X_val, y_val), callbacks = cd, verbose = 1)

In [ ]:
#Plot the graphical interpretation
h = hist.history
plt.plot(h['accuracy'], label = 'accuracy')
plt.plot(h['val_accuracy'], label = 'val-accuracy')
plt.title('Accuracy vs Val Accuracy')
plt.legend()
plt.show()

In [ ]:
#Plot the graphical interpretation
h = hist.history
plt.plot(h['loss'], label = 'loss')
plt.plot(h['val_loss'], label = 'val-loss')
plt.title('Loss vs Val Loss')
plt.legend()
plt.show()


In [ ]:
#Test our model on the test set
from keras.models import load_model
model = load_model('/content/bestmodel.h5')
acc = model.evaluate(X_test, y_test)[1]
print(f'The accuracy of our model is {acc}')

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf

# Choose a MRI image with tumor
path = "/content/aug_data/yes/aug_Y25_0_5988.jpg"
img_yes = Image.open(path).resize((240,240))
img_array_yes = np.array(img_yes)/255
img_array_yes = np.expand_dims(img_array_yes, axis=0)

# Load the trained model
model = tf.keras.models.load_model('bestmodel.h5')

# Make a prediction on the image
prediction1 = model.predict(img_array_yes)
class1 = np.round(prediction1).astype(int)
if class1 == 0:
  print("The MRI image doesn't have a Tumor")
else:
  print("The MRI image has a tumor")

In [ ]:
from PIL import Image

# Choose an MRI image without tumor
path = "/content/aug_data/no/aug_no 10_0_7554.jpg"
img_no = Image.open(path).resize((240, 240))
img_array_no = np.array(img_no)/255
img_array_no = np.expand_dims(img_array_no, axis=0)
prediction2 = model.predict(img_array_no)
class2 = np.round(prediction2).astype(int)
if class2 == 0:
    print("The MRI image doesn't have a tumor")
else:
    print("The MRI image has a tumor")

In [ ]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
# Load VGG16 model with imagenet weights and exclude the top layer
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(240, 240, 3))

# Freeze all the layers in the base model
for layer in vgg_model.layers:
    layer.trainable = False

# Add a custom top layer
model = Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()



In [ ]:
# Set up ModelCheckpoint to save the best model based on validation accuracy
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

# Train the model with checkpoint
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[checkpoint,early_stop])

# Load the best model
model.load_weights('best_model.h5')

In [ ]:
h = history.history
plt.plot(h['accuracy'], label = 'accuracy')
plt.plot(h['val_accuracy'], label = 'val-accuracy')
plt.title('Accuracy vs Val Accuracy')
plt.legend()
plt.show()

In [ ]:
#Plot the graphical interpretation
h = history.history
plt.plot(h['loss'], label = 'loss')
plt.plot(h['val_loss'], label = 'val-loss')
plt.title('Loss vs Val Loss')
plt.legend()
plt.show()

In [ ]:
#Test our model on the test set
from keras.models import load_model
model = load_model('/content/best_model.h5')
acc = model.evaluate(X_test, y_test)[1]
print(f'The accuracy of our model is {acc}')

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf

# Choose a MRI image with tumor
path = "/content/aug_data/yes/aug_Y6_0_767.jpg"
img_yes = Image.open(path).resize((240,240))
img_array_yes = np.array(img_yes)/255
img_array_yes = np.expand_dims(img_array_yes, axis=0)

# Load the trained model
model = tf.keras.models.load_model('best_model.h5')

# Make a prediction on the image
prediction1 = model.predict(img_array_yes)
class1 = np.round(prediction1).astype(int)
if class1 == 0:
  print("The MRI image doesn't have a Tumor")
else:
  print("The MRI image has a tumor")

In [ ]:
from PIL import Image

# Choose an MRI image without tumor
path = "/content/aug_data/no/aug_5 no_0_6712.jpg"
img_no = Image.open(path).resize((240, 240))
img_array_no = np.array(img_no)/255
img_array_no = np.expand_dims(img_array_no, axis=0)
prediction2 = model.predict(img_array_no)
class2 = np.round(prediction2).astype(int)
if class2 == 0:
    print("The MRI image doesn't have a tumor")
else:
    print("The MRI image has a tumor")

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

model = load_model('/content/best_model.h5')
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Convert predicted values to binary values using threshold of 0.5
y_test_pred = np.where(y_test_pred >= 0.5, 1, 0)

# Generate confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

# Plot confusion matrix as heatmap
df_cm = pd.DataFrame(cm, index=[i for i in ["1",'0']], columns=[i for i in ["1","0"]])
plt.figure(figsize=(4,3))
sns.heatmap(df_cm, annot=True, fmt='g')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, matthews_corrcoef
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
acc= (tp+tn)/ (tp+tn+fp+fn)
pre= tp/(fp+tp)
rec=tp/(fn+tp)
f1 = 2*pre*rec/(pre+rec)
specificity = tn / (tn + fp)
fpr = fp / (tn + fp)
fnr = fn / (fn + tp)
npv = tn / (tn + fn)
fdr = fp / (fp + tp)
mcc = matthews_corrcoef(y_test, y_test_pred)
print('VGG16 Model additional performance metrics for Test set')
print('- Accuracy: {:.4f}'.format(acc))
print('- Precision: {:.4f}'.format(pre))
print('- Recall: {:.4f}'.format(rec))
print('- Specificity: {:.4f}'.format(specificity))
print('- F1 score: {:.4f}'.format(f1))
print('- FPR: {:.4f}'.format(fpr))
print('- FNR: {:.4f}'.format(fnr))
print('- NPV: {:.4f}'.format(npv))
print('- FDR: {:.4f}'.format(fdr))
print('- MCC: {:.4f}'.format(mcc))

In [ ]:
from keras.applications import VGG19
from keras.models import Sequential
from keras.layers import Dense, Flatten

# Load VGG19 model with imagenet weights and exclude the top layer
vgg_model = VGG19(weights='imagenet', include_top=False, input_shape=(240, 240, 3))

# Freeze all the layers in the base model
for layer in vgg_model.layers:
    layer.trainable = False

# Add a custom top layer
model = Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
# Set up ModelCheckpoint to save the best model based on validation accuracy
checkpoint = ModelCheckpoint('bestt_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

# Train the model with checkpoint
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[checkpoint,early_stop])

# Load the best model
model.load_weights('bestt_model.h5')

In [ ]:
h = history.history
plt.plot(h['accuracy'], label = 'accuracy')
plt.plot(h['val_accuracy'], label = 'val-accuracy')
plt.title('Accuracy vs Val Accuracy')
plt.legend()
plt.show()

In [ ]:
#Plot the graphical interpretation
h = history.history
plt.plot(h['loss'], label = 'loss')
plt.plot(h['val_loss'], label = 'val-loss')
plt.title('Loss vs Val Loss')
plt.legend()
plt.show()

In [ ]:
#Test our model on the test set
from keras.models import load_model
model = load_model('/content/bestt_model.h5')
acc = model.evaluate(X_test, y_test)[1]
print(f'The accuracy of our model is {acc}')

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf

# Choose a MRI image with tumor
path = "/content/aug_data/yes/aug_Y248_0_5054.jpg"
img_yes = Image.open(path).resize((240,240))
img_array_yes = np.array(img_yes)/255
img_array_yes = np.expand_dims(img_array_yes, axis=0)

# Load the trained model
model = tf.keras.models.load_model('bestt_model.h5')

# Make a prediction on the image
prediction1 = model.predict(img_array_yes)
class1 = np.round(prediction1).astype(int)
if class1 == 0:
  print("The MRI image doesn't have a Tumor")
else:
  print("The MRI image has a tumor")

In [ ]:
from PIL import Image

# Choose an MRI image without tumor
path = "/content/aug_data/no/aug_7 no_0_1966.jpg"
img_no = Image.open(path).resize((240, 240))
img_array_no = np.array(img_no)/255
img_array_no = np.expand_dims(img_array_no, axis=0)
prediction2 = model.predict(img_array_no)
class2 = np.round(prediction2).astype(int)
if class2 == 0:
    print("The MRI image doesn't have a tumor")
else:
    print("The MRI image has a tumor")

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

model = load_model('/content/bestt_model (2).h5')
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Convert predicted values to binary values using threshold of 0.5
y_test_pred = np.where(y_test_pred >= 0.5, 1, 0)

# Generate confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

# Plot confusion matrix as heatmap
df_cm = pd.DataFrame(cm, index=[i for i in ["1",'0']], columns=[i for i in ["1","0"]])
plt.figure(figsize=(4,3))
sns.heatmap(df_cm, annot=True, fmt='g')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, matthews_corrcoef
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
acc= (tp+tn)/ (tp+tn+fp+fn)
pre= tp/(fp+tp)
rec=tp/(fn+tp)
f1 = 2*pre*rec/(pre+rec)
specificity = tn / (tn + fp)
fpr = fp / (tn + fp)
fnr = fn / (fn + tp)
npv = tn / (tn + fn)
fdr = fp / (fp + tp)
mcc = matthews_corrcoef(y_test, y_test_pred)
print('VGG19 Model additional performance metrics for Test set')
print('- Accuracy: {:.4f}'.format(acc))
print('- Precision: {:.4f}'.format(pre))
print('- Recall: {:.4f}'.format(rec))
print('- Specificity: {:.4f}'.format(specificity))
print('- F1 score: {:.4f}'.format(f1))
print('- FPR: {:.4f}'.format(fpr))
print('- FNR: {:.4f}'.format(fnr))
print('- NPV: {:.4f}'.format(npv))
print('- FDR: {:.4f}'.format(fdr))
print('- MCC: {:.4f}'.format(mcc))

In [ ]:
from keras.applications import MobileNetV2
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D

# Load MobileNetV2 model with imagenet weights and exclude the top layer
mobilenet_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(240, 240, 3))

# Freeze all the layers in the base model
for layer in mobilenet_model.layers:
    layer.trainable = False

# Add a custom top layer
model = Sequential()
model.add(mobilenet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Set up ModelCheckpoint to save the best model based on validation accuracy
checkpoint = ModelCheckpoint('best1_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

# Train the model with checkpoint
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[checkpoint,early_stop])

# Load the best model
model.load_weights('best1_model.h5')

In [ ]:
h = history.history
plt.plot(h['accuracy'], label = 'accuracy')
plt.plot(h['val_accuracy'], label = 'val-accuracy')
plt.title('Accuracy vs Val Accuracy')
plt.legend()
plt.show()

In [ ]:
#Plot the graphical interpretation
h = history.history
plt.plot(h['loss'], label = 'loss')
plt.plot(h['val_loss'], label = 'val-loss')
plt.title('Loss vs Val Loss')
plt.legend()
plt.show()

In [ ]:
#Test our model on the test set
from keras.models import load_model
model = load_model('/content/best1_model.h5')
acc = model.evaluate(X_test, y_test)[1]
print(f'The accuracy of our model is {acc}')

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf

# Choose a MRI image with tumor
path = "/content/aug_data/yes/aug_Y81_0_4331.jpg"
img_yes = Image.open(path).resize((240,240))
img_array_yes = np.array(img_yes)/255
img_array_yes = np.expand_dims(img_array_yes, axis=0)

# Load the trained model
model = tf.keras.models.load_model('best1_model.h5')

# Make a prediction on the image
prediction1 = model.predict(img_array_yes)
class1 = np.round(prediction1).astype(int)
if class1 == 0:
  print("The MRI image doesn't have a Tumor")
else:
  print("The MRI image has a tumor")

In [ ]:
from PIL import Image

# Choose an MRI image without tumor
path = "/content/aug_data/no/aug_14 no_0_9400.jpg"
img_no = Image.open(path).resize((240, 240))
img_array_no = np.array(img_no)/255
img_array_no = np.expand_dims(img_array_no, axis=0)
prediction2 = model.predict(img_array_no)
class2 = np.round(prediction2).astype(int)
if class2 == 0:
    print("The MRI image doesn't have a tumor")
else:
    print("The MRI image has a tumor")

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

model = load_model('/content/best1_model.h5')
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Convert predicted values to binary values using threshold of 0.5
y_test_pred = np.where(y_test_pred >= 0.5, 1, 0)

# Generate confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

# Plot confusion matrix as heatmap
df_cm = pd.DataFrame(cm, index=[i for i in ["1",'0']], columns=[i for i in ["1","0"]])
plt.figure(figsize=(4,3))
sns.heatmap(df_cm, annot=True, fmt='g')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, matthews_corrcoef
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
acc= (tp+tn)/ (tp+tn+fp+fn)
pre= tp/(fp+tp)
rec=tp/(fn+tp)
f1 = 2*pre*rec/(pre+rec)
specificity = tn / (tn + fp)
fpr = fp / (tn + fp)
fnr = fn / (fn + tp)
npv = tn / (tn + fn)
fdr = fp / (fp + tp)
mcc = matthews_corrcoef(y_test, y_test_pred)
print('MobileNetV2 Model additional performance metrics for Test set')
print('- Accuracy: {:.4f}'.format(acc))
print('- Precision: {:.4f}'.format(pre))
print('- Recall: {:.4f}'.format(rec))
print('- Specificity: {:.4f}'.format(specificity))
print('- F1 score: {:.4f}'.format(f1))
print('- FPR: {:.4f}'.format(fpr))
print('- FNR: {:.4f}'.format(fnr))
print('- NPV: {:.4f}'.format(npv))
print('- FDR: {:.4f}'.format(fdr))
print('- MCC: {:.4f}'.format(mcc))

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Activation

base_Neural_Net= ResNet50(input_shape=(240,240,3), weights='imagenet', include_top=False)
model=Sequential()
model.add(base_Neural_Net)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

for layer in base_Neural_Net.layers:
    layer.trainable = False


model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Set up ModelCheckpoint to save the best model based on validation accuracy
checkpoint = ModelCheckpoint('best2_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

# Train the model with checkpoint
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[checkpoint,early_stop])

# Load the best model
model.load_weights('best2_model.h5')

In [ ]:
h = history.history
plt.plot(h['accuracy'], label = 'accuracy')
plt.plot(h['val_accuracy'], label = 'val-accuracy')
plt.title('Accuracy vs Val Accuracy')
plt.legend()
plt.show()

In [ ]:
#Plot the graphical interpretation
h = history.history
plt.plot(h['loss'], label = 'loss')
plt.plot(h['val_loss'], label = 'val-loss')
plt.title('Loss vs Val Loss')
plt.legend()
plt.show()

In [ ]:
#Test our model on the test set
from keras.models import load_model
model = load_model('/content/best2_model.h5')
acc = model.evaluate(X_test, y_test)[1]
print(f'The accuracy of our model is {acc}')

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf

# Choose a MRI image with tumor
path = "/content/aug_data/yes/aug_Y81_0_4331.jpg"
img_yes = Image.open(path).resize((240,240))
img_array_yes = np.array(img_yes)/255
img_array_yes = np.expand_dims(img_array_yes, axis=0)

# Load the trained model
model = tf.keras.models.load_model('best2_model.h5')

# Make a prediction on the image
prediction1 = model.predict(img_array_yes)
class1 = np.round(prediction1).astype(int)
if class1 == 0:
  print("The MRI image doesn't have a Tumor")
else:
  print("The MRI image has a tumor")

In [ ]:
from PIL import Image

# Choose an MRI image without tumor
path = "/content/aug_data/no/aug_14 no_0_9400.jpg"
img_no = Image.open(path).resize((240, 240))
img_array_no = np.array(img_no)/255
img_array_no = np.expand_dims(img_array_no, axis=0)
prediction2 = model.predict(img_array_no)
class2 = np.round(prediction2).astype(int)
if class2 == 0:
    print("The MRI image doesn't have a tumor")
else:
    print("The MRI image has a tumor")

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

model = load_model('/content/best2_model.h5')
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Convert predicted values to binary values using threshold of 0.5
y_test_pred = np.where(y_test_pred >= 0.5, 1, 0)

# Generate confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

# Plot confusion matrix as heatmap
df_cm = pd.DataFrame(cm, index=[i for i in ["1",'0']], columns=[i for i in ["1","0"]])
plt.figure(figsize=(4,3))
sns.heatmap(df_cm, annot=True, fmt='g')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, matthews_corrcoef
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
acc= (tp+tn)/ (tp+tn+fp+fn)
pre= tp/(fp+tp)
rec=tp/(fn+tp)
f1 = 2*pre*rec/(pre+rec)
specificity = tn / (tn + fp)
fpr = fp / (tn + fp)
fnr = fn / (fn + tp)
npv = tn / (tn + fn)
fdr = fp / (fp + tp)
mcc = matthews_corrcoef(y_test, y_test_pred)
print('ResNet50 Model additional performance metrics for Test set')
print('- Accuracy: {:.4f}'.format(acc))
print('- Precision: {:.4f}'.format(pre))
print('- Recall: {:.4f}'.format(rec))
print('- Specificity: {:.4f}'.format(specificity))
print('- F1 score: {:.4f}'.format(f1))
print('- FPR: {:.4f}'.format(fpr))
print('- FNR: {:.4f}'.format(fnr))
print('- NPV: {:.4f}'.format(npv))
print('- FDR: {:.4f}'.format(fdr))
print('- MCC: {:.4f}'.format(mcc))

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet101V2

base_Neural_Net= ResNet101V2(input_shape=(240,240,3), weights='imagenet', include_top=False)
model=Sequential()
model.add(base_Neural_Net)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

for layer in base_Neural_Net.layers:
    layer.trainable = False


model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Set up ModelCheckpoint to save the best model based on validation accuracy
checkpoint = ModelCheckpoint('best3_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

# Train the model with checkpoint
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[checkpoint,early_stop])

# Load the best model
model.load_weights('best3_model.h5')

In [ ]:
h = history.history
plt.plot(h['accuracy'], label = 'accuracy')
plt.plot(h['val_accuracy'], label = 'val-accuracy')
plt.title('Accuracy vs Val Accuracy')
plt.legend()
plt.show()

In [ ]:
#Plot the graphical interpretation
h = history.history
plt.plot(h['loss'], label = 'loss')
plt.plot(h['val_loss'], label = 'val-loss')
plt.title('Loss vs Val Loss')
plt.legend()
plt.show()

In [ ]:
#Test our model on the test set
from keras.models import load_model
model = load_model('/content/best3_model.h5')
acc = model.evaluate(X_test, y_test)[1]
print(f'The accuracy of our model is {acc}')

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf

# Choose a MRI image with tumor
path = "/content/aug_data/yes/aug_Y81_0_4331.jpg"
img_yes = Image.open(path).resize((240,240))
img_array_yes = np.array(img_yes)/255
img_array_yes = np.expand_dims(img_array_yes, axis=0)

# Load the trained model
model = tf.keras.models.load_model('best3_model.h5')

# Make a prediction on the image
prediction1 = model.predict(img_array_yes)
class1 = np.round(prediction1).astype(int)
if class1 == 0:
  print("The MRI image doesn't have a Tumor")
else:
  print("The MRI image has a tumor")

In [ ]:
from PIL import Image

# Choose an MRI image without tumor
path = "/content/aug_data/no/aug_14 no_0_9400.jpg"
img_no = Image.open(path).resize((240, 240))
img_array_no = np.array(img_no)/255
img_array_no = np.expand_dims(img_array_no, axis=0)
prediction2 = model.predict(img_array_no)
class2 = np.round(prediction2).astype(int)
if class2 == 0:
    print("The MRI image doesn't have a tumor")
else:
    print("The MRI image has a tumor")

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

model = load_model('/content/best3_model.h5')
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Convert predicted values to binary values using threshold of 0.5
y_test_pred = np.where(y_test_pred >= 0.5, 1, 0)

# Generate confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

# Plot confusion matrix as heatmap
df_cm = pd.DataFrame(cm, index=[i for i in ["1",'0']], columns=[i for i in ["1","0"]])
plt.figure(figsize=(4,3))
sns.heatmap(df_cm, annot=True, fmt='g')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, matthews_corrcoef
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
acc= (tp+tn)/ (tp+tn+fp+fn)
pre= tp/(fp+tp)
rec=tp/(fn+tp)
f1 = 2*pre*rec/(pre+rec)
specificity = tn / (tn + fp)
fpr = fp / (tn + fp)
fnr = fn / (fn + tp)
npv = tn / (tn + fn)
fdr = fp / (fp + tp)
mcc = matthews_corrcoef(y_test, y_test_pred)
print('ResNet101 v2 Model additional performance metrics for Test set')
print('- Accuracy: {:.4f}'.format(acc))
print('- Precision: {:.4f}'.format(pre))
print('- Recall: {:.4f}'.format(rec))
print('- Specificity: {:.4f}'.format(specificity))
print('- F1 score: {:.4f}'.format(f1))
print('- FPR: {:.4f}'.format(fpr))
print('- FNR: {:.4f}'.format(fnr))
print('- NPV: {:.4f}'.format(npv))
print('- FDR: {:.4f}'.format(fdr))
print('- MCC: {:.4f}'.format(mcc))

In [ ]:
import matplotlib.pyplot as plt

# Define a list of model names and their corresponding accuracy values
model_names = ['CNN', 'VGG16', 'VGG19', 'MOBILENET V2','RESNET50','RESNET101 V2']
accuracy_values = [0.9097, 0.9799, 0.985, 0.9924,0.9323,0.9924]

fig, ax = plt.subplots(figsize=(10, 5))

# Plot the graph
barplot = ax.bar(model_names, accuracy_values,width=0.5)
ax.set_title('Model Accuracy Comparison')
ax.set_xlabel('Model Name')
ax.set_ylabel('Accuracy')
ax.set_ylim(0.8, 1.0)

# Add the accuracy values above the bars
for bar, acc in zip(barplot, accuracy_values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height()+0.001, f'{acc:.3f}', ha='center')
plt.show()
